# Chapter 08 웹 크롤링

## 8.2 웹 데이터 자동 수집

### HTML 소스 가져오기

In [ ]:
!pip install requests

### 파이썬 웹 크롤링 패키지
#### [ requests ]
- 브라우저 없이 URL에 접속, 결과값(HTML)을 추출함
- 웹 브라우저를 실행하지 않고 특정 URL에 요청(request)을 보낼 수 있음
- 응답(response)된 데이터의 헤더(Header), 바디(HTML) 등의 정보 확인
- 빠르고 안정적이지만 동적 웹 페이지를 크롤링하기 위해서는 많은 분석이 필요

#### [ selenium ]
- 브라우저를 자동으로 실행해 URL에 접속, 결과값(HTML)을 추출함
- 시뮬레이션하듯이 웹 브라우저를 직접 실행하여 URL에 접속할 수 있음
- 응답된 데이터의 헤더(Header), 바디(HTML) 등의 정보를 확인할 수 있음
- 느리고 불안정하지만 동적 웹 페이지 크롤링이 쉽고 초보자가 사용하기 좋음

#### [ BeautifulSoup ] 
- requests와 selenium으로 받아온 내용(HTML)을 분석
- requests나 selenium을 통해 얻은 HTML 데이터를 파싱(parsing)해 객체로 구조화
- HTML 수집은 앞선 두 모듈이 담당하지만, 분석은 BeautifulSoup이 담당
- HTML 태그와 CSS 속성을 이용해 원하는 정보를 찾을 수 있는 기능 제공

In [ ]:
import requests                                              # requests 모듈 불러오기
res = requests.get("http://www.naver.com", verify = False)   # GET 방식으로 네이버 홈페이지 가져오기
print(res.content)                                            # HTML 형식의 데이터 출력

# verify를 False로 지정하면 SSL인증서 오류 문제가 발생해도 URL에 입력한 웹 페이지를 가져올 수 있다.

--------------

In [ ]:
# GET 방식
parameters = {"code" : "005930"}     # 삼성전자 종목코드
res = requests.get("https://finance.naver.com/item/sise.nhn", params=parameters, verify = False)
print(res.url)
print(res.text)    # 한글 출력이 가능하여 텍스트 위주의 웹 페이지일 경우 유용하게 사용

In [ ]:
res = requests.get("https://finance.naver.com/item/sise.nhn?code=005930", verify = False)
print(res.text)

In [ ]:
# POST 방식
res = requests.post("https://finance.naver.com/item/sise.nhn = 005930", verify = False)
print(res.text)

### HTML 소스를 데이터로 변환하기

In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup                    # BeautifulSoup 모듈 불러오기

# html_txt 변수에 HTML 소스와 텍스트 입력
html_txt = "<p class='wether' id='tw'>오늘의 날씨</p>  <h1> 한때 소나기가 내리겠습니다.</h1>"

soup = BeautifulSoup(html_txt,"html.parser")      # 파싱하기, 분석 가능한 데이터 형태로 변환

In [ ]:
tag = soup.find("p")        # p 태그를 찾아 tag에 저장

print(tag)                  # tag 출력
print(tag.name)             # tag명 출력
print(tag.attrs )           # tag 속성 출력
print(tag.attrs["class"])   # tag 속성 중 class만 출력
print(tag.attrs["id"])      # tag 속성 중 id만 출력
print(tag.text)             # tag 내 텍스트 출력

In [ ]:
from bs4 import BeautifulSoup

# HTML 소스 생성
html_txt = """                
<html>
<head><title>BS page</title></head>
<body>

<h1 class="portal_cls">검색포털</h1>
<p>
<a href="http://www.daum.net">다음 바로가기</a><br>
<a href="http://www.naver.com">네이버 바로가기</a>
</p>
<h1 class="portal_cls">검색엔진</h1>
<a href="http://www.google.com" class="alink_cls">구글</a>
<p class="footage_cls" id="company">2021, ABC Company </p>
<p class="footage_cls" id="addr">Korea</p>
</body>
</html>
"""
soup = BeautifulSoup(html_txt, "html.parser")       # 파싱하기, 분석 가능한 데이터 형태로 변환

In [ ]:
# 특정 태그 찾기
tag = soup.select_one("h1")                 # <h1> </h1> 태그 추출
print(tag.text)                             # tag 안에 있는 텍스트만 출력

In [ ]:
tag_list = soup.select("h1")                # <h1> ~ </h1> 태그 모두 찾기
for tag in tag_list:                       # tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)                        # 텍스트만 출력

In [ ]:
# 자식/자손 태그 찾기
tag_list = soup.select("body p > a")       # <body><p> 태그 하위의 <a> 태그 추출
for tag in tag_list:                       # tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)                       # 텍스트만 출력

In [ ]:
# CSS ID로 태그 찾기
tag_list = soup.select(".footage_cls")    # footage_cls 클래스 추출
for tag in tag_list:                     # tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)                      # 텍스트만 출력

In [ ]:
# 태그 속성값을 이용해서 태그 찾기
tag_list = soup.select("a[href]")        # a[href] 추출
for tag in tag_list:                    # tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text, tag.attrs["href"])  # 텍스트와 URL 출력

### [실습] 서점 베스트셀러 정보 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

res = requests.get("http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?range = \1&kind=2&orderClick = DAB&mallGb=KOR&linkClass=B")
html = res.content
book = bs(html, "html.parser")

book_list = []
for book_detail in book.select("div.info"):
    book_urls = book_detail.select_one("div.title > a > span").attrs["href"]
    book_list.append(book_urls)

In [ ]:
book_list

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

res = requests.get("https://product.kyobobook.co.kr/bestseller/total#?page=1&per=20&period=002&ymw=&bsslBksClstCode=B")
html = res.content
book = bs(html, "html.parser")

In [ ]:
print(res.content)

In [ ]:
# 패키지 불러오기
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# 교보문고의 월간 베스트셀러(소설) 웹 페이지 가져오기
res = requests.get("http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?range = \1&kind=2&orderClick = DAB&mallGb=KOR&linkClass=B")
html = res.content                              # 웹 리소스 가져오기
book = bs(html, "html.parser")                  # HTML 파싱

# 도서 상세 페이지 주소 추출하기
book_list = []                                  # book_list 리스트 변수 만들기
for book_detail in book.select("div.detail"):  # book에서 class=detail 태그 가져오기
    book_urls = book_detail.select_one("div.title > a").attrs["href"]   # 상세 주소를 book_urls에 저장
    book_list.append(book_urls)                 # book_list 변수에 주소를 하나씩 추가

# 도서별 상세 데이터 추출하기
best = pd.DataFrame(columns=["도서","저자","가격","url"])     # 저장할 데이터프레임 만들기
for index, book_url in enumerate(book_list):                 # 도서 상세 페이지를 하나씩 반환
    res = requests.get(book_url)                             # 도서 상세 페이지 리소스 가져오기
    html = res.content
    best_book = bs(html, "html.parser", from_encoding = "cp949")        # HTML 파싱
    title = best_book.select_one("h1.title strong").text.strip()        # 도서명 추출
    author = best_book.select_one("span.name a").text.strip()           # 저자명 추출
    price = best_book.select_one("span.sell_price strong").text.strip() # 가격 추출    
    best.loc[index+1] = (title, author, price, book_url)                # 데이터 프레임에 저장

# 엑셀 파일로 저장하기
best.to_excel("bestseller.xlsx")

## 8.3 웹 브라우저 제어

### selenium 개요 

In [ ]:
# !pip install selenium

### 웹 로드 및 HTML 소스 가져오기

In [ ]:
import selenium                      # selenium 모듈 불러오기
from selenium import webdriver      # webdriver 모듈 불러오기

In [ ]:
URL = "https://www.naver.com"               # 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")   # driver 객체 생성 
driver.get(URL)                             # URL에 저장된 주소를 크롬에서 열기

In [ ]:
print(driver.current_url)				# 현재 URL 출력

In [ ]:
driver.close()

In [ ]:
import selenium                              # selenium 모듈 불러오기
from bs4 import BeautifulSoup               # BeautifulSoup 모듈 불러오기
from selenium import webdriver              # webdriver 모듈 불러오기
URL = "https://www.naver.com"                # 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")    # driver 객체 생성 
driver.get(URL)                              # URL에 저장된 주소를 크롬에서 열기

html = driver.page_source                    # HTML 소스 가져오기
soup = BeautifulSoup(html, "html.parser")    # 데이터 변환(파싱)

tag_list = soup.select("body p")             # <body> 내 <p> 태그 추출
for tag in tag_list:                        # tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)                         # 텍스트만 출력
driver.close()    

### 웹 브라우저 제어하기

In [ ]:
import selenium                               # selenium 모듈 불러오기
from selenium import webdriver               # webdriver 모듈 불러오기
from selenium.webdriver.common.by import By  # selenium의 by 클래스 불러오기
URL = "https://www.naver.com"                 # 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")     # driver 객체 생성 
driver.get(URL)                               # URL에 저장된 주소를 크롬에서 열기
btn = driver.find_element(By.XPATH, '//*[@id="account"]/a')  # 네이버 로그인 버튼 선택
btn.click()                                   # 로그인 버튼 클릭

In [ ]:
import selenium                              # selenium 모듈 불러오기
from selenium import webdriver              # webdriver 모듈 불러오기
from selenium.webdriver.common.by import By # selenium의 by 클래스 불러오기
URL = "https://www.naver.com"                # 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")    # driver 객체 생성 
driver.get(URL)                              # URL에 저장된 주소를 크롬에서 열기
input_tag = driver.find_element(By.CSS_SELECTOR, "#query")  # 검색창으로 이동
input_tag.send_keys("박효신")            # 검색창에 문자 입력
input_tag.send_keys("\n")                    # Enter 키 실행
driver.implicitly_wait(3)                    # 최대 3초간 대기(3초가 지나면 다음 코드 수행)

### [실습] 네이버 환율 정보 수집 후 CSV로 저장하기

In [ ]:
# 패키지 불러오기
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

# 네이버 금융 웹 페이지로 이동하기
driver = webdriver.Chrome("chromedriver.exe")       # 크롬 드라이버 실행
url = "https://finance.naver.com/marketindex"       # 사이트 URL
driver.get(url)                                     # URL에 저장된 주소를 크롬에서 열기
driver.switch_to.frame("frame_ex1")                 # 환율 정보 프레임으로 전환

# 파싱 및 태그 추출하기
html = driver.page_source                           # 페이지 정보 가져오기
soup = BeautifulSoup(html, "html.parser")           # HTML 파싱하기
result = []                                         # 리스트 변수 생성
currencys = soup.select("body > div > table > tbody > tr")   # selector로 정보 가져오기

for data in currencys :
    country = data.select("td.tit > a")[0].text.strip()      # 통화명 가져오기
    exchange = data.select("td.sale")[0].text                # 매매기준율 가져오기
    result.append([country, exchange])                       # 리스트 변수에 저장

# CSV 파일 만들기
df = pd.DataFrame(result, columns = ["통화명", "환율"])                       # 표 컬럼 항목 추가
df.to_csv("환율정보.csv", encoding = "cp949", header = True, index = False)  # CSV 파일 생성
driver.close()                                                                 # 웹 드라이버 종료

### [실습] 사업자등록번호 휴폐업 조회하기

In [ ]:
# 패키지 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# 사업자등록번호 조회 웹 페이지 접속하기
driver = webdriver.Chrome("chromedriver")       # chromedriver.exe 파일 위치
url ="https://teht.hometax.go.kr/websquare/websquare.html?w2xPath=/ui/ab/a/a/UTEABAAA13.xml"
driver.get(url)                                 # 브라우저 열기
time.sleep(3)                                   # 3초 대기

# 사업자등록번호 엑셀 파일 읽어오기
resultList = []                                 # 결과 저장 리스트 변수 생성
df = pd.read_excel("사업자등록번호.xlsx")       # 엑셀 파일 불러오기

# 사업자등록번호를 입력해서 결과값 가져오기
for regNo in df["Business_no"] :                # for 문 생성
    driver.find_element(By.CSS_SELECTOR, "#bsno").send_keys(regNo)  # 사업자등록번호 입력
    driver.find_element(By.CSS_SELECTOR, "#trigger5").click()       # 조회하기 버튼 클릭
    time.sleep(2)                               # 2초 대기시간 유지 
    result = driver.find_element(By.CSS_SELECTOR, "#grid2_cell_0_1").text      # 결과값 가져오기
    resultList.append(result)                   # 결과값을 reslutList에 저장하기

# 조회 결과 저장하기
df["result"] = resultList                       # df에 열 추가
df.to_excel("사업자등록번호 조회 결과.xlsx", encoding="cp949", index = False) # 엑셀로 저장
driver.close()                                  # 드라이버 종료